# Graphical Elastic Net — Python Analysis Notebook
This notebook ingests the CSVs from the reproduction kit (`results/scores_model_*.csv`),summarizes the performance of **glasso (α=1)**, **rope (α=0)**, and **gelnet (α=0.5)**,and produces publication-friendly plots (matplotlib only).**Usage**: Place this notebook in the kit root and run after executing `R/reproduce_simulations.R`.Outputs:- Aggregated tables (per-model, per-method, per-target, ±diag-penalization)- Deltas vs. glasso/rope baselines- Composite ranking (z-normalized metrics)- Matplotlib figures under `plots_py/`

In [29]:
import os, glob, math
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

KIT_ROOT = Path('.')
RES_DIR = KIT_ROOT  / 'data' / 'extended_experiments'
PLOT_DIR = KIT_ROOT / 'plots_py'
PLOT_DIR.mkdir(exist_ok=True)
#csv_files = sorted(RES_DIR.glob('scores_model_*.csv'))
csv_files = glob.glob(str(RES_DIR / 'processed_results.csv'))
if not csv_files:
    print('No simulation CSVs found under results/. Run R/reproduce_simulations.R first.')
else:
    print(f'Found {len(csv_files)} CSV files.')

Found 1 CSV files.


In [30]:
def load_scores(files):
    dfs = []    
    for fp in files:
        try:
            df = pd.read_csv(fp)
            dfs.append(df)
        except Exception as e:
            print(f'Failed to read {fp}: {e}')
            if not dfs:
                return pd.DataFrame()
    return pd.concat(dfs, ignore_index=True)

scores = load_scores(csv_files)
scores.head(3) if not scores.empty else scores

,model,rep,method,alpha,penalize_diag,target,lambda,KL,L2,SP,edges,F1,MCC,TP,TN,FP,FN,PRAUC,ROCAUC
0,6,24,glasso,0.20,True,NaN,0.125893,290.787953,717.612391,160.301729,470,0.326870,0.294934,118,4346,352,134,0.300003,0.703888
1,6,24,glasso,0.35,True,NaN,0.118850,290.137293,717.525143,160.281550,550,0.316708,0.289518,127,4275,423,125,0.298559,0.716390
2,6,24,glasso,0.50,True,NaN,0.125893,290.787953,717.612391,160.301729,470,0.326870,0.294934,118,4346,352,134,0.300003,0.703888


## Clean-up and sanity checks- Ensure factor types; derive convenient labels- Infer effective target label (already baked into the R kit, but we keep it explicit)

In [31]:
if scores.empty:
    print('No data loaded. Skip the rest of the notebook until results exist.')
else:
    scores['model'] = scores['model'].astype(int)
    scores['rep'] = scores['rep'].astype(int)
    scores['penalize_diag'] = scores['penalize_diag'].astype(bool)    # Readable labels    scores['pen_label'] = np.where(scores['penalize_diag'], 'diag✔', 'diag✘')    scores['method_target'] = scores['method'] + ' | ' + scores['target'] + ' | ' + scores['pen_label']    # Sanity    print(scores.groupby(['model','method']).size().head())

## Aggregate metricsWe compute mean and std for KL, L2, SP (↓ is better), and F1, MCC (↑ is better).

In [32]:
if not scores.empty:
    agg = (scores
           .groupby(['model','method','target','penalize_diag'], as_index=False)
           .agg({                'KL':['mean','std'],                'L2':['mean','std'],
                 'SP':['mean','std'],
                 'F1':['mean','std'],
                 'MCC':['mean','std']
                }))    # flatten columns    agg.columns = ['_'.join([c for c in map(str, col) if c and c!='None']) for col in agg.columns.values]    agg.head()else:    agg = pd.DataFrame()    agg

## Baseline deltasFor each model, compute deltas vs **glasso (α=1, target=None, diag✔)** and **rope (α=0, target=None, diag✔)**.Negative delta is better for KL/L2/SP; positive delta is better for F1/MCC.

In [33]:
def pick_baseline(agg, model, method_name):
    rows = agg[(agg['model']==model) & (agg['method']==method_name) &
    (agg['target']=='None') & (agg['penalize_diag']==True)]
    return rows.iloc[0] if len(rows)>0 else None

def D(name, base):
    return None if base is None else row[name]- base[name]

if not agg.empty:
    deltas = []
    for m in sorted(agg['model'].unique()):
        g_base = pick_baseline(agg, m, 'glasso')
        r_base = pick_baseline(agg, m, 'rope')
        for _, row in agg[agg['model']==m].iterrows():
            d = row.to_dict()                
            for metric in ['KL_mean','L2_mean','SP_mean']:
                d[f'delta_{metric}_vs_glasso'] = D(metric, g_base)
                d[f'delta_{metric}_vs_rope']   = D(metric, r_base)
            for metric in ['F1_mean','MCC_mean']:
                d[f'delta_{metric}_vs_glasso'] = D(metric, g_base)
                d[f'delta_{metric}_vs_rope']   = D(metric, r_base)
            deltas.append(d)
else:
    deltas = pd.DataFrame()
    deltas

deltas = pd.DataFrame(deltas)
deltas.head()

,"(model, )","(method, )","(target, )","(penalize_diag, )","(KL, mean)","(KL, std)","(L2, mean)","(L2, std)","(SP, mean)","(SP, std)",...,delta_KL_mean_vs_glasso,delta_KL_mean_vs_rope,delta_L2_mean_vs_glasso,delta_L2_mean_vs_rope,delta_SP_mean_vs_glasso,delta_SP_mean_vs_rope,delta_F1_mean_vs_glasso,delta_F1_mean_vs_rope,delta_MCC_mean_vs_glasso,delta_MCC_mean_vs_rope
0,1,gelnet,Eigenvalue,True,116.540240,26.237285,13.289149,3.305848,2.740033,0.522381,...,None,None,None,None,None,None,None,None,None,None
1,1,gelnet,Identity,True,38.355616,5.770517,6.408969,0.597011,1.359886,0.087564,...,None,None,None,None,None,None,None,None,None,None
2,1,gelnet,MSC,True,39.962132,5.957650,5.955649,0.520634,1.266502,0.074755,...,None,None,None,None,None,None,None,None,None,None
3,1,gelnet,Regression,True,55.748269,18.080053,5.554235,0.442481,1.166938,0.088283,...,None,None,None,None,None,None,None,None,None,None
4,1,gelnet,TrueDiag,True,103.850507,16.422496,3.798649,0.978216,0.848410,0.165870,...,None,None,None,None,None,None,None,None,None,None


## Composite rankCreate a composite score per (model, method, target, diag) by z-normalizing metrics with the right directionality:- For KL/L2/SP: lower is better (use -z)- For F1/MCC: higher is better (use +z)

In [34]:
def zsafe(s: pd.Series) -> pd.Series:
    # coerce to numeric, keep index, handle all-NaN or zero-variance cases
    s = pd.to_numeric(s, errors='coerce')
    mu = np.nanmean(s.values) if s.notna().any() else 0.0
    sd = np.nanstd(s.values) if s.notna().any() else 0.0
    if not np.isfinite(sd) or sd == 0.0:
        # avoid exploding/NaN z-scores; return zeros (neutral contribution)
        return pd.Series(np.zeros(len(s)), index=s.index)
    return (s - mu) / (sd + 1e-12)

if not agg.empty:
    rows = []
    # group once; don’t mutate `comp` inside the loop
    for m, sub in agg.groupby('model', sort=True):
        sub = sub.copy()

        # Robust MultiIndex selection with tuples:
        KLm  = sub[('KL',  'mean')]
        L2m  = sub[('L2',  'mean')]
        SPm  = sub[('SP',  'mean')]
        F1m  = sub[('F1',  'mean')]
        MCCm = sub[('MCC', 'mean')]

        # Lower-better: KL, L2, SP → negative sign; higher-better: F1, MCC → positive
        sub['score'] = (
            -zsafe(KLm) + -zsafe(L2m) + -zsafe(SPm) + zsafe(F1m) + zsafe(MCCm)
        )

        # Higher score ranks better; tie-breaking “min” as you had
        sub['rank'] = sub['score'].rank(method='min', ascending=False)

        rows.append(sub)

    comp = pd.concat(rows, ignore_index=True)
    comp = comp.sort_values(['model', 'rank'], kind='mergesort')  # stable sort
    # show top-10 per model (optional)
    top = comp.groupby('model', group_keys=False).apply(lambda df: df.nsmallest(10, 'rank'))
else:
    comp = pd.DataFrame()
    top = comp

def z(x):
    return (x - np.nanmean(x)) / (np.nanstd(x) + 1e-12)
if not agg.empty:
    comp = agg.copy()
    rows = []
    for m in sorted(comp['model'].unique()):
        sub = comp[comp['model']==m].copy()
        print(sub)
        sub['score'] = (-z(sub[('KL','mean')]) + -z(sub[('L2','mean')]) + -z(sub[('SP','mean')]) + z(sub[('F1','mean')]) + z(sub[('MCC','mean')]))
        sub['rank'] = (-sub['score']).rank(method='min')
        rows.append(sub)
        comp = pd.concat(rows, ignore_index=True)
        comp.sort_values(['model','rank']).head(10)
else:
    comp = pd.DataFrame()
    comp


   model  method      target penalize_diag          KL                    L2  \
                                                  mean        std       mean   
0      1  gelnet  Eigenvalue          True  116.540240  26.237285  13.289149   
1      1  gelnet    Identity          True   38.355616   5.770517   6.408969   
2      1  gelnet         MSC          True   39.962132   5.957650   5.955649   
3      1  gelnet  Regression          True   55.748269  18.080053   5.554235   
4      1  gelnet    TrueDiag          True  103.850507  16.422496   3.798649   
5      1  gelnet   vIdentity          True   38.323791   5.776053   6.407294   
6      1  glasso  Eigenvalue          True   80.766907  10.652478   8.723980   
7      1  glasso    Identity          True   43.028574   6.143519   6.210777   
8      1  glasso         MSC          True   43.009449   6.062038   6.206194   
9      1  glasso  Regression          True   45.749245   6.518457   6.201954   
10     1  glasso    TrueDiag          Tr

/tmp/ipykernel_46453/248136357.py:37: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  top = comp.groupby('model', group_keys=False).apply(lambda df: df.nsmallest(10, 'rank'))
/tmp/ipykernel_46453/248136357.py:37: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top = comp.groupby('model', group_keys=False).apply(lambda df: df.nsmallest(10, 'rank'))
/tmp/ipykernel_46453/248136357.py:43: RuntimeWarning: Mean of empty slice
  return (x - np.nanmean(x)) / (np.nanstd(x) + 1e-12)
/home/immanuel/dev/PhD-studies/.venv/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2015: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(

## PlotsMatplotlib-only, one metric per figure, one figure per model to keep outputs clean.

In [35]:

def _resolve_series(df, key):
    """Return a Series for either ('KL','mean') or 'KL_mean' style keys."""
    if isinstance(key, tuple) and key in df.columns:
        return df[key]
    if isinstance(key, str):
        if key in df.columns:
            return df[key]
        if "_" in key:
            a, b = key.split("_", 1)
            tup = (a, b)
            if tup in df.columns:
                return df[tup]
    raise KeyError(f"Metric column {key!r} not found.")

def _key_to_fname(key):
    return "_".join(key) if isinstance(key, tuple) else key

def barplot_metric(comp_df, metric_key, ylabel, fname):
    # Ensure x labels exist (method • target [+ diag flag])
    if 'method_target' not in comp_df.columns:
        diag_flag = np.where(comp_df.get('penalize_diag', False), ' (diag✔)', ' (diag✘)')
        comp_df = comp_df.assign(
            method_target = comp_df['method'].astype(str) + ' • ' + comp_df['target'].astype(str) + diag_flag
        )
    models = sorted(comp_df['model'].unique())

    for m in models:
        sub = comp_df.loc[comp_df['model'] == m].copy()
        if sub.empty:
            continue

        # stable order for readability
        sub = sub.sort_values(['method','target','penalize_diag'], kind='mergesort')

        y = _resolve_series(sub, metric_key).astype(float).fillna(np.nan)
        x = np.arange(len(sub))

        plt.figure(figsize=(max(6, len(sub)*0.35), 4.5))
        plt.bar(x, y.values)
        plt.xticks(x, sub['method_target'], rotation=90)
        plt.ylabel(ylabel)
        plt.title(f'Model {m} — {ylabel}')
        plt.tight_layout()

        out = PLOT_DIR / f"model_{m}_{_key_to_fname(fname)}.png"
        plt.savefig(out, dpi=160)
        plt.close()
        print('Saved', out)

# ---- call it ----
if not agg.empty:
    # choose keys that work for a MultiIndex (('METRIC','mean')) or flat ('METRIC_mean')
    metric_specs = [
        (('KL','mean'),  'KL',       'KL_mean'),
        (('L2','mean'),  'L2',       'L2_mean'),
        (('SP','mean'),  'Spectral', 'SP_mean'),
        (('F1','mean'),  'F1',       'F1_mean'),
        (('MCC','mean'), 'MCC',      'MCC_mean'),
    ]
    for key, lab, fname in metric_specs:
        barplot_metric(agg, key, lab, fname)


#f barplot_metric(comp_df, metric, ylabel, fname):
#    models = sorted(comp_df['model'].unique())
#    for m in models:
#        sub = comp_df[comp_df['model']==m].copy()
#        if sub.empty:
#            continue
#        x = np.arange(len(sub))
#        plt.figure()
#        plt.bar(x, sub[metric].values)
#        plt.xticks(x, sub['method_target'], rotation=90)
#        plt.ylabel(ylabel)
#        plt.title(f'Model {m} — {ylabel}')
#        plt.tight_layout()
#        out = PLOT_DIR / f'model_{m}_{fname}.png'
#        plt.savefig(out, dpi=160)
#        plt.close()
#        print('Saved', out)
#if not agg.empty:
#    print(agg)
#    for metric, lab in [(['KL']['mean'],'KL'), ('L2_mean','L2'), ('SP_mean','Spectral'), ('F1_mean','F1'), ('MCC_mean','MCC')]:
#            barplot_metric(agg, metric, lab, f'{metric}')

Saved plots_py/model_1_KL_mean.png
Saved plots_py/model_2_KL_mean.png
Saved plots_py/model_3_KL_mean.png
Saved plots_py/model_4_KL_mean.png
Saved plots_py/model_5_KL_mean.png
Saved plots_py/model_6_KL_mean.png
Saved plots_py/model_1_L2_mean.png
Saved plots_py/model_2_L2_mean.png
Saved plots_py/model_3_L2_mean.png
Saved plots_py/model_4_L2_mean.png
Saved plots_py/model_5_L2_mean.png
Saved plots_py/model_6_L2_mean.png
Saved plots_py/model_1_SP_mean.png
Saved plots_py/model_2_SP_mean.png
Saved plots_py/model_3_SP_mean.png
Saved plots_py/model_4_SP_mean.png
Saved plots_py/model_5_SP_mean.png
Saved plots_py/model_6_SP_mean.png
Saved plots_py/model_1_F1_mean.png
Saved plots_py/model_2_F1_mean.png
Saved plots_py/model_3_F1_mean.png
Saved plots_py/model_4_F1_mean.png
Saved plots_py/model_5_F1_mean.png
Saved plots_py/model_6_F1_mean.png
Saved plots_py/model_1_MCC_mean.png
Saved plots_py/model_2_MCC_mean.png
Saved plots_py/model_3_MCC_mean.png
Saved plots_py/model_4_MCC_mean.png
Saved plots_py/m

## Save key tables

In [36]:
if not agg.empty:
    agg.to_csv(PLOT_DIR / 'aggregate_metrics.csv', index=False)
    deltas.to_csv(PLOT_DIR / 'deltas_vs_baselines.csv', index=False)
    comp.to_csv(PLOT_DIR / 'composite_rank.csv', index=False)
    print('Saved aggregate tables to plots_py/.')

Saved aggregate tables to plots_py/.
